In [1]:
import os
import sys
import cv2
import PIL
import time
import shutil
import numpy as np
import tensorflow as tf
from PIL import Image,ImageTk
from keras.preprocessing.image import ImageDataGenerator
from keras.applications.vgg16 import VGG16
import pytesseract
from pytesseract import Output

In [2]:
from keras.layers import Dense,Flatten
from keras.models import Sequential, load_model


In [3]:
pytesseract.pytesseract.tesseract_cmd = r'C:\Program Files\Tesseract-OCR\tesseract.exe'

In [4]:
location = input("Enter the location of the image : ")
image = cv2.imread(location,0)
thresh = cv2.threshold(image, 150, 255, cv2.THRESH_BINARY_INV)[1]

Enter the location of the image : sentences/say.png


In [5]:
result = cv2.GaussianBlur(thresh, (5,5), 0)
result = 255 - result

In [6]:
cv2.imwrite('final_img.png',result)
loc = 'final_img.png'

In [7]:
if os.path.isfile('DATA/train/.DS_Store'):
    os.remove('DATA/train/.DS_Store')
if os.path.isfile('DATA/test/.DS_Store'):
    os.remove('DATA/test/.DS_Store')
    
for i in range(1,80):
    if os.path.isfile(f'DATA/train/{i}/.DS_Store'):
        os.remove(f'DATA/train/{i}/.DS_Store')
    if os.path.isfile(f'DATA/test/{i}/.DS_Store'):
        os.remove(f'DATA/test/{i}/.DS_Store')

In [8]:
datagen = ImageDataGenerator(rescale = 1./255,
                            zoom_range = 0.2)

In [9]:
trained_image = datagen.flow_from_directory('DATA/train',
                                            target_size = (32,32),
                                            class_mode = 'categorical')

Found 650 images belonging to 26 classes.


In [10]:
test_datagen = ImageDataGenerator(rescale = 1./255)

In [11]:
test_image = test_datagen.flow_from_directory('DATA/test',
                                            target_size = (32,32),
                                            class_mode = 'categorical')

Found 130 images belonging to 26 classes.


In [11]:
conv_base = VGG16(weights='vgg16_weights_tf_dim_ordering_tf_kernels_notop.h5', include_top = False, input_shape = (32,32,3))

In [12]:
def Train_Model():
    global trained_image, test_image
    model = Sequential()
    #conv_base = VGG16(weights='imagenet', include_top = False, input_shape = (32,32,3))
    model.add(conv_base)
    model.add(Flatten())
    model.add(Dense(256,activation='relu'))
    model.add(Dense(128,activation='relu'))
    model.add(Dense(26,activation = 'softmax'))
    model.compile(optimizer= 'adam', loss = 'categorical_crossentropy', metrics = ['accuracy'])
    model.fit(trained_image, epochs = 30, validation_data = test_image, validation_steps = 1)
    model.save('model.h5')
    
Train_Model()

Epoch 1/30
21/21 [==============================] - 29s 1s/step - loss: 3.3305 - accuracy: 0.0277 - val_loss: 3.3969 - val_accuracy: 0.0000e+00
Epoch 2/30
21/21 [==============================] - 26s 1s/step - loss: 3.2831 - accuracy: 0.0292 - val_loss: 3.2615 - val_accuracy: 0.0312
Epoch 3/30
21/21 [==============================] - 26s 1s/step - loss: 3.2044 - accuracy: 0.0492 - val_loss: 2.9365 - val_accuracy: 0.0938
Epoch 4/30
21/21 [==============================] - 27s 1s/step - loss: 2.9019 - accuracy: 0.0738 - val_loss: 2.7367 - val_accuracy: 0.0625
Epoch 5/30
21/21 [==============================] - 26s 1s/step - loss: 2.5411 - accuracy: 0.1015 - val_loss: 2.1515 - val_accuracy: 0.2500
Epoch 6/30
21/21 [==============================] - 26s 1s/step - loss: 2.3465 - accuracy: 0.1692 - val_loss: 2.1277 - val_accuracy: 0.0625
Epoch 7/30
21/21 [==============================] - 27s 1s/step - loss: 1.9723 - accuracy: 0.2615 - val_loss: 1.6611 - val_accuracy: 0.3750
Epoch 8/30
21/21

In [12]:
def Predict_Model(img):
    global trained_image
    new_model = load_model('model.h5')
    img = cv2.resize(img,(32,32),3)
    img = np.expand_dims(img,axis=0)
    img = img / 255
    letters = ['a','b','c','d','e','f','g','h','i','j','k','l','m','n','o','p','q','r','s','t','u','v','w','x','y','z','A','B','C','D','E','F','G','H','I','J','K','L','M','N','O','P','Q','R','S','T','U','V','W','X','Y','Z','1','2','3','4','5','6','7','8','9','0',',',';',':','?','!','.','@','#','$','%','&','(',')','{','}','[',']']
    
    #prediction = new_model.predict_classes(img)
    predict_x=new_model.predict(img) 
    
    classes_x=np.argmax(predict_x,axis=1)
    
    prediction = classes_x[0]
    
    my_dict = dict(trained_image.class_indices)
    
    for key,value in my_dict.items():
        if prediction == value:
            return key

In [14]:
def Word_Extract(location):
    
    if os.path.isdir('WORDS'):
        shutil.rmtree('WORDS')
        os.mkdir('WORDS')
    else:
        os.mkdir('WORDS')
    img = PIL.Image.open(location)
    d = pytesseract.image_to_data(Image.open(location) , output_type=Output.DICT)
    n_boxes = len(d['level'])
    select =[]
    j =0
    for i in range(n_boxes):
        if d['text'][i] != '' and d['conf'][i] != '-1':
            select.append(d['text'][i])
            (x, y, w, h) = (d['left'][i]-20, d['top'][i]-20, d['width'][i]+20, d['height'][i]+20)
            #cv2.rectangle(img, (x, y), (x + w, y + h), (0, 255, 0), 2)
            crop_img = img.crop((x,y,x+w,y+h))
            crop_img.save(f'WORDS/{j}.png')
            j += 1
    
    return  select
      
    
select = Word_Extract(loc)

In [15]:
if os.path.isdir('LETTERS'):
    shutil.rmtree('LETTERS')
    os.mkdir('LETTERS')
else:
    os.mkdir('LETTERS')
    
    
for i in range(len(os.listdir('WORDS'))):
    os.mkdir(f'LETTERS/{i}')
    img = Image.open(f"WORDS/{i}.png")
    img = img.resize((1600 , 800))
    w,h = img.size
    letters = pytesseract.image_to_boxes(img , output_type=Output.DICT)
    letters
    
    idx = 0 
    for c in range(len(letters['char'])): 
        
        (x, y, w, h) = (letters['left'][c], letters['bottom'][c], letters['right'][c], letters['top'][c])
        
        crop_img = img.crop((x-50,y-50 , w+50,h+50))
        crop_img.save(f"LETTERS/{i}/" + str(idx) + '.png')
        idx+=1
            

In [16]:
text = ''
for i in range(len(os.listdir('LETTERS'))):
    string = ''
    char_dict = {}
    for j in range(len(os.listdir(f'LETTERS/{i}'))):
        img = cv2.imread(f'LETTERS/{i}/{j}.png')
        img = cv2.cvtColor(img,cv2.COLOR_BGR2RGB)
        char = Predict_Model(img)
        
        
        for l in range(len(select[i])):
            if char == select[i][l]:
                char_dict[select[i].index(char)] = char
                
                
            else:
                for k in range(len(select[i])):
                    if k != l:
                        if char == select[i][k]:
                            char_dict[k] = char 
        
        keys = list(char_dict.keys())
        if char not in select[i]:
            for val in range(len(select[i])):
                if val not in keys:
                    char_dict[val] = char
    keys = list(char_dict.keys())
    keys.sort()
    #print(keys)
    
    for m in keys:
        string += char_dict[m]
        
        
    string += ' '
    text += string
              

In [17]:
for i in range(10):
    print("")
print("The predicted sentence is: ",text)
    











The predicted sentence is:  AKARSH MALIK 
